In [2]:
import sys
sys.path.append("../")

import src.dataset_manager as dataset_manager

In [3]:
dataset_manager.DatasetManager.list_dataset_groups()

['sst2', 'geometry_of_truth']

In [4]:
dataset_manager.DatasetManager.list_datasets_by_group()

{'geometry_of_truth': ['sp_en_trans',
  'neg_sp_en_trans',
  'cities',
  'neg_cities',
  'smaller_than',
  'larger_than',
  'common_claim_true_false',
  'companies_true_false',
  'counterfact_true_false'],
 'sst2': ['sst2']}

In [9]:
custom_dataset = dataset_manager.DatasetManager.from_dataset_group("geometry_of_truth", 2)

In [10]:
len(custom_dataset)

22637

In [11]:
for batch in custom_dataset:
    print(batch)
    break

[RawExample(feature='Ninety-five is smaller than eighty-eight.', label='0'), RawExample(feature='Picasso once cut off part of each of triangular-shaped ear before being taken to a hospital.', label='0')]
